In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2023-11-05 17:43:26--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8002::154, 2606:50c0:8001::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8002::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  --.-KB/s    in 0.09s   

2023-11-05 17:43:26 (11.6 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [2]:
with open('input.txt') as f:
    data = f.read()

In [3]:
charset = sorted(list(set(data)))
vocab_size=  len(charset)

print(''.join(charset),vocab_size,sep='\n')



 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [11]:
# encode and decode using serialization
stoi = {ch:i for i,ch in enumerate(charset)}
itos = {i:ch for i,ch in enumerate(charset)}

encoder  = lambda s: [stoi[ch] for ch in s]
decoder = lambda n: [itos[i] for i in n]


encoded = encoder(data[:10])
decoded = decoder(encoded)
print(data[:10],encoded,decoded,sep="\n")

First Citi
[18, 47, 56, 57, 58, 1, 15, 47, 58, 47]
['F', 'i', 'r', 's', 't', ' ', 'C', 'i', 't', 'i']


In [4]:
import tiktoken

In [16]:
# encoder = tiktoken.encoding_for_model("gpt-4")

# enc = encoder.encode(data[:10])
# dec = encoder.decode(enc)

# print(enc,dec)

In [6]:
import torch
import numpy as np

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [12]:
tkns = torch.tensor(encoder(data))

tkns

tensor([18, 47, 56,  ..., 45,  8,  0])

In [10]:
tkns

tensor([ 5451, 47317,   512,  ...,  1989, 48728,   627])

In [10]:
n = int(0.9*len(tkns))
train_data = tkns[:n+1]
val_data = tkns[n+1:]

In [13]:
torch.manual_seed(0)
batch_size = 4
block_size = 8
ix = torch.randint(len(tkns)-batch_size,(batch_size,))

def get_batch(batch_size,block_size,data_split = "train"):
    data = train_data if data_split == "train" else val_data
    ix = torch.randint(len(tkns)-block_size,(batch_size,))
    input_data = torch.stack([data[i:i+block_size] for i in ix])
    target = torch.stack([data[i+1:i+1+block_size] for i in ix])

    return input_data,target

def visualize(input_data,target):
    for ix in range(len(input_data)):
        for i in range(block_size):
            inpt = input_data[ix,:i+1]
            otpt = target[ix,i]
            print('when input is {}, output is {}'.format(inpt,otpt))

i,o = get_batch(batch_size=batch_size,block_size=block_size)
print(i)
print(o)

# print(i.shape,i[1,:])
visualize(i,o)




tensor([[58, 46, 43, 56,  1, 51, 43,  6],
        [54, 53, 47, 57, 53, 52, 43, 56],
        [33, 15, 23, 21, 26, 19, 20, 13],
        [21, 17, 32, 10,  0, 37, 53, 52]])
tensor([[46, 43, 56,  1, 51, 43,  6,  1],
        [53, 47, 57, 53, 52, 43, 56,  0],
        [15, 23, 21, 26, 19, 20, 13, 25],
        [17, 32, 10,  0, 37, 53, 52,  1]])
when input is tensor([58]), output is 46
when input is tensor([58, 46]), output is 43
when input is tensor([58, 46, 43]), output is 56
when input is tensor([58, 46, 43, 56]), output is 1
when input is tensor([58, 46, 43, 56,  1]), output is 51
when input is tensor([58, 46, 43, 56,  1, 51]), output is 43
when input is tensor([58, 46, 43, 56,  1, 51, 43]), output is 6
when input is tensor([58, 46, 43, 56,  1, 51, 43,  6]), output is 1
when input is tensor([54]), output is 53
when input is tensor([54, 53]), output is 47
when input is tensor([54, 53, 47]), output is 57
when input is tensor([54, 53, 47, 57]), output is 53
when input is tensor([54, 53, 47, 57,

In [13]:
i

tensor([[11604,   198,  2409,   293,  1604,  1285, 70535,   323],
        [  499,  2643,   617,  9160,  2454,   264,   628,  5827],
        [49972,   382,    43,  5576,  3895,  1473, 81361,   537],
        [  345,  3112, 15411,  1057, 13162,   304,  3776, 76350]])

In [14]:
i[-1][-1]

tensor(76350)

In [14]:
import torch.nn
# vocab_size = encoder.n_vocab

embeddings = torch.nn.Embedding(vocab_size,vocab_size)

emb = embeddings(i)

In [18]:
emb.shape

tensor([[[ 0.0714,  0.8801,  0.1912,  ...,  0.1745,  0.9324,  0.0393],
         [ 1.6723,  0.5173,  0.9703,  ..., -0.6564,  1.7701, -0.5448],
         [ 1.0838, -0.0849, -1.6914,  ...,  0.6126, -1.4797,  0.3492],
         ...,
         [ 2.3328, -1.3859,  1.0154,  ..., -1.1096,  1.1374, -0.9502],
         [ 1.0838, -0.0849, -1.6914,  ...,  0.6126, -1.4797,  0.3492],
         [ 0.6816,  1.5178, -0.6353,  ...,  0.1027, -0.4830, -0.2608]],

        [[-1.9330,  1.1050, -0.1408,  ..., -0.0831, -1.6705, -0.4293],
         [-1.0829, -0.9296,  1.5269,  ..., -0.4009, -0.3732, -0.7805],
         [ 1.3584,  0.4848,  1.2366,  ...,  0.8587, -2.4105,  0.8027],
         ...,
         [-1.5136, -1.1765, -0.7252,  ..., -0.1077, -0.0091,  0.7436],
         [ 1.0838, -0.0849, -1.6914,  ...,  0.6126, -1.4797,  0.3492],
         [ 0.3418, -0.2136, -0.2575,  ...,  0.8734, -1.5082, -2.1618]],

        [[-1.2984, -0.1949, -0.4208,  ..., -1.0695, -2.0217,  0.7896],
         [ 1.2397, -1.0817, -0.3302,  ..., -0

In [17]:
print(embeddings(torch.tensor([64])).view(-1))


tensor([-0.9801,  0.6824, -0.7586, -0.5210,  0.3097, -0.8427,  1.3237, -0.7553,
        -0.3777, -0.6922, -0.6532, -0.2703,  1.9668, -1.4623,  0.2717, -1.1529,
        -0.0772,  0.2818,  1.5907,  0.2737,  0.8273, -0.6517, -2.5686,  1.3187,
         0.3632, -0.7837,  0.5977,  0.6139,  1.9707, -0.2860, -0.8189,  0.6200,
        -2.7965,  0.3460,  0.3316, -0.9157, -0.7333,  0.6930, -1.4625, -0.9355,
        -0.5180, -1.1246,  0.6485,  0.7721,  0.3665,  1.2456, -0.3826, -0.1085,
        -0.0827, -0.8512, -0.2678,  0.8035, -0.4030, -0.8973, -0.6216, -0.5001,
        -0.9062, -1.4205, -0.0802, -0.1577,  0.0168, -1.3619,  0.8723, -0.1968,
        -0.1024], grad_fn=<ViewBackward0>)


In [73]:
import torch.nn.functional as F
from torch import nn
class BigramModel(nn.Module):
    def __init__(self):
        super().__init__()
        vocab_size = 65
        self.embedding = torch.nn.Embedding(vocab_size,vocab_size)
    
    def forward(self,inpts,targets=None):
        logits = self.embedding(inpts)
        if targets is None:
            loss = None
        else:
            # print(logits)
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)

            #loss_fcn = torch.nn.CrossEntropyLoss()
            #loss = loss_fcn(target, logits)
            loss = F.cross_entropy(logits,targets)
        return logits,loss
    
    def generate_tkns(self,inpts,max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(inpts)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            inpts = torch.cat((inpts, idx_next), dim=1) # (B, T+1)
        return inpts






In [76]:
m = BigramModel()
print(m(i,o))
m.generate_tkns(inpts = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()
print(''.join(decoder(m.generate_tkns(inpts = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist())))

(tensor([[ 0.3858, -0.4780,  0.4837,  ...,  1.4251, -0.9275,  0.8669],
        [-0.2404, -0.4747, -0.4338,  ..., -0.7907,  1.8102,  0.8979],
        [ 0.2246,  1.5730, -0.6026,  ...,  0.2024, -0.1892,  0.6314],
        ...,
        [-0.1428,  0.1078, -1.4439,  ...,  1.3444,  4.9990, -0.1206],
        [ 1.6918, -0.6464,  0.9655,  ..., -1.4093,  0.5757,  0.6651],
        [-0.8869,  2.1616, -1.1989,  ..., -2.0739, -1.0880,  2.3583]],
       grad_fn=<ViewBackward0>), tensor(4.3023, grad_fn=<NllLossBackward0>))

wx-,&wBuVEoorPbNHy?L3v
heXZJ$3uc:
RU
RQfnzNDM!fnYxZpfi:JT-VD
PZyZN
OxvGpUQs?3pHThO'I&kQW.AXwjsESndyy


In [47]:
m

BigramModel(
  (embedding): Embedding(65, 65)
)